In [1]:
#from  scapy.all import *
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
from scapy.all import *
#files name in pcap direcoty 
import glob

In [2]:
#Take all split values from folder, that I just done above

In [3]:
split_files = glob.glob("NF_othr/*.pcap")

In [4]:
split_files

['NF_othr/1_host_2_elastic_3.pcap',
 'NF_othr/1_host_2_exponential_7.pcap',
 'NF_othr/1_host_2_elastic_7.pcap',
 'NF_othr/1_host_2_average_1.pcap',
 'NF_othr/2_host_2_average_2.pcap',
 'NF_othr/1_host_2_logistic_8.pcap',
 'NF_othr/1_host_2_elastic_5.pcap',
 'NF_othr/2_host_2_elastic_4.pcap',
 'NF_othr/1_host_2_exponential_4.pcap',
 'NF_othr/2_host_2_elastic_2.pcap',
 'NF_othr/1_host_2_exponential_3.pcap',
 'NF_othr/2_host_2_logistic_1.pcap',
 'NF_othr/2_host_2_conventional_1.pcap',
 'NF_othr/2_host_2_conventional_4.pcap',
 'NF_othr/1_host_2_elastic_2.pcap',
 'NF_othr/1_host_2_exponential_8.pcap',
 'NF_othr/2_host_2_elastic_8.pcap',
 'NF_othr/1_host_2_conventional_3.pcap',
 'NF_othr/2_host_2_logistic_7.pcap',
 'NF_othr/2_host_2_logistic_4.pcap',
 'NF_othr/1_host_2_average_3.pcap',
 'NF_othr/2_host_2_exponential_2.pcap',
 'NF_othr/2_host_2_average_6.pcap',
 'NF_othr/2_host_2_exponential_9.pcap',
 'NF_othr/2_host_2_conventional_7.pcap',
 'NF_othr/1_host_2_average_9.pcap',
 'NF_othr/2_host

In [5]:
total_split_files = len(split_files)

In [6]:
total_split_files

90

In [7]:
names = split_files[0].split('.')


In [8]:
node_number = names[0].split('/')

In [9]:
node_number[1]

'1_host_2_elastic_3'

In [12]:
for s_files in range(total_split_files):
    print(split_files[s_files])
    pcap =rdpcap(split_files[s_files])
    reset= []
    for p in pcap:
        if p.haslayer(TCP) and (p[TCP].flags == 'RA' or p[TCP].flags == 'R') : #reset,ack or reset flag RA mean both reset and ack
            if p[IP].src == '192.168.0.150': #server port
                #print("2015-----")
                #print(p[TCP].sport)
                #print(p[TCP].dport)
                reset.append(p[TCP].dport)
            else:
                #print("not 2015---")
                reset.append(p[TCP].sport)
                #print(p[TCP].sport)
                #print(p[TCP].dport)
            
    #head= []

    #for p in pcap:
        #if p.haslayer(TCP): 
            #if Raw in p[TCP]:
                #raw = str(p[Raw])
                #if 'HEAD' in raw:
                    #head.append(p[TCP].sport)   

#print(head)


    connection=[]; connection_sip=[]; connection_dip=[]; connection_sport=[]; connection_sip=[]
    c=0
    pcap[1].show()  
    extra=0; segg=1; array=[]; array2=[]
    for ppp in pcap:
        # Select full connection except reset and head connection 
        if (ppp.haslayer(TCP)) and ( ppp[TCP].sport not in reset):
            #print('fuu'+str(ppp[TCP].sport))
            if Raw in ppp[TCP]:
                if(ppp[IP].dst == '192.168.0.150'):# server ip
                    if 'GET' in str(ppp[Raw]):
                        connection.append(c)
                        c=c+1
                        connection_sip.append(ppp[IP].src)
                        connection_sport.append(ppp[TCP].sport)
                        if ("sintel_enc_x264_dash.mpd" in str(ppp[Raw]) or "sintel_enc_x264_dash_init.mp4"  in str(ppp[Raw])):
                                extra=extra+1
                        #elif 'segment'+str(segg) in str(ppp[Raw]):
                         #           array.append(ppp[TCP].sport)
                          #          segg=segg+1
                        #else:
                            #array2.append(segg)
                            #segg=segg+1
                            #continue
                        #print(array)
                        #print(segg)
    #print(extra)                  
    conn=[] ;ij=extra-1;ik=0            
    conn=sorted(list(set(connection_sport)))                    
    for q in range(len(conn)-extra):
         for qq in pcap:
            if  qq.haslayer(TCP) and ((qq[TCP].sport == conn[q+extra])):
                 if Raw in qq[TCP]:
                    if(qq[IP].dst == '192.168.0.150'):# server ip
                        if 'GET' in str(qq[Raw]):
                                print(str((q+extra)-ij) )
                                if ((q+extra)-ij)>59:
                                    break
                                if 'segment'+str((q+extra)-ij) in str(qq[Raw]):
                                    array.append(qq[TCP].sport)
                                    print(str((q+extra)-ij) )
                                    print("conn:"+str(conn[(q+extra)-ij]))
                                    #print(array)
                                    segg=segg+1
                                    #ij=3
                                #print(segg)
                                #perint
                                else:
                                    ik=ik+1
                                    array2.append((q+extra)-ij)
                                    ij=extra-(1+ik)
    df= pd.DataFrame(columns=['Port','Segment','RTT','Throughput', 'Packets'])# rtt=uplink ms, th=downlink byte per sec, sp= downlink packets

    t1=0;t2=0;l=0;inner =True;k=0;b=True;s=0;

    i=1; 
    length= (len(connection))-extra
    print(extra)
    print(length)
    m=extra
    for j in range(len(conn)-m):
        pac1= True;pac= True; ttt=True;f=True; c=0;tc= True; r1=0; initrtt=0; r2=0;  inner=True; inner2=False;httprtt=0; aaa=0;summ=0;l=0;tspop=0
        for p in pcap:
            if p.haslayer(TCP) and (p[TCP].sport == conn[j+m] or p[TCP].dport== conn[j+m]):
                if pac:
                    p1_t=p.time
                    pac=False
                if(p[IP].dst == connection_sip[1]):  
                    if(  p[TCP].flags == 'SA'):
                        if(tc==True):
                            p2_t= p.time
                            initrtt= (p2_t - p1_t) * 1000
                            print("initrtt" + str(initrtt))
                            tc=False

                if  p[IP].src == connection_sip[j+m] and pac1 and  Raw in p[TCP]:
                                                    #if (segment in str(p[Raw])):
                    r1 = p.time
                    print('r1---'+ str(r1))
                    pac1 = False
                    #print(p[IP].src)

                if p[IP].dst == connection_sip[j+m]:

                        if Raw in p[TCP]:
                            l=l+int(len(p[Raw]))
                            if len(p[TCP]) > 32:
                                    pcakets=p[TCP]
                                    tstop=p.time
                                    c=c+1  

                        if ttt and (p[TCP].flags == 'A' or p[TCP].flags == 'PA'):
                            r2=p.time
                            httprtt= r2 - r1
                            print('r2---'+ str(r2))
                            aaa= httprtt * 1000
                            summ= (initrtt+aaa)/2
                            ttt=False    

                if p[TCP].flags == 'FA':# 
                    th=float(l/(tstop-r1))
                    if j+m in array2:
                        i=i+1
                        continue


                    #df.loc[i]=[aaa ,th ,c ]
                    df = df.append(pd.Series([conn[j+m], i, aaa ,th ,c], index=df.columns ), ignore_index=True)
                    i=i+1
                    #print(df)

                    break
    names = split_files[s_files].split('.')

    node_number = names[0].split('/')
        
    
    export_csv = df.to_csv ('NF_othr/qos_othr/'+node_number[1]+'.csv', index = None, header=True) #Don't forget   
    print('Done') 


NF_othr/1_host_2_elastic_3.pcap
###[ Ethernet ]### 
  dst       = 36:5c:75:9c:b8:e2
  src       = e6:16:23:ee:df:08
  type      = ARP
###[ ARP ]### 
     hwtype    = 0x1
     ptype     = IPv4
     hwlen     = 6
     plen      = 4
     op        = is-at
     hwsrc     = e6:16:23:ee:df:08
     psrc      = 192.168.0.150
     hwdst     = 36:5c:75:9c:b8:e2
     pdst      = 192.168.0.1

1
1
conn:33962
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
2
20
initrtt12.891000
r1---1591336636.487124
r2---1591336636.487255
initrtt12.884000
r1---1591336636.514590
r2---1591336636.514734
initrtt31.170000
r1---1591336638.649454
r2---1591336638.689256
initrtt27.343000
r1---1591336640.128381
r2---1591336640.142972
initrtt48.855000
r1---1591336641.290598
r2---1591336641.317603
initrtt26.697000
r1---1591336643.451429
r2---1591336643.463782
initrtt31.350000
r1---1591336645.527829
r2---1591336645.554956
initrtt29.442000
r1---1591336647.730441
r2---1591336647.742784
initrtt41.664000
r1---1591336649.580274

initrtt17.654000
r1---1591370870.023417
r2---1591370870.045574
initrtt43.607000
r1---1591370871.979620
r2---1591370872.000957
initrtt22.939000
r1---1591370873.695598
r2---1591370873.714700
initrtt16.573000
r1---1591370875.355954
r2---1591370875.364906
initrtt26.974000
r1---1591370877.109170
r2---1591370877.122935
initrtt26.905000
r1---1591370878.884922
r2---1591370878.898501
initrtt20.011000
r1---1591370880.678929
r2---1591370880.692009
initrtt18.703000
r1---1591370882.625842
r2---1591370882.636390
initrtt17.268000
r1---1591370884.307301
r2---1591370884.322565
initrtt26.321000
r1---1591370886.127005
r2---1591370886.140638
initrtt27.859000
r1---1591370887.992791
r2---1591370888.010879
initrtt36.062000
r1---1591370890.121544
r2---1591370890.139154
initrtt35.207000
r1---1591370892.096274
r2---1591370892.115949
initrtt19.880000
r1---1591370893.950656
r2---1591370893.959608
initrtt29.305000
r1---1591370895.667366
r2---1591370895.695214
initrtt35.686000
r1---1591370897.527021
r2---1591370897

21
21
conn:34310
22
22
conn:34318
23
23
conn:34326
24
24
conn:34334
25
25
conn:34342
26
26
conn:34350
27
27
conn:34358
28
28
conn:34366
29
29
conn:34374
30
30
conn:34382
31
31
conn:34390
32
32
conn:34398
33
33
conn:34402
34
34
conn:34412
35
35
conn:34420
36
36
conn:34428
37
37
conn:34434
38
38
conn:34442
39
39
conn:34452
40
40
conn:34460
41
41
conn:34464
42
42
conn:34474
43
43
conn:34484
44
44
conn:34492
45
45
conn:34496
46
46
conn:34508
47
47
conn:34516
48
48
conn:34526
49
49
conn:34530
50
50
conn:34542
51
51
conn:34548
52
52
conn:34554
53
53
conn:34566
54
54
conn:34570
55
55
conn:34582
56
56
conn:34590
57
57
conn:34598
58
58
conn:34606
59
59
conn:34614
60
0
60
initrtt7.828000
r1---1591281084.659832
r2---1591281084.659922
initrtt26.890000
r1---1591281085.454009
r2---1591281085.482937
initrtt21.494000
r1---1591281086.689631
r2---1591281086.713970
initrtt25.650000
r1---1591281088.957291
r2---1591281088.967579
initrtt22.535000
r1---1591281091.701476
r2---1591281091.724458
initrtt25.85400

initrtt21.822000
r1---1591371887.852544
r2---1591371887.863366
initrtt25.251000
r1---1591371889.484549
r2---1591371889.487044
initrtt24.567000
r1---1591371890.943507
r2---1591371890.945964
initrtt25.984000
r1---1591371892.530680
r2---1591371892.532239
initrtt17.636000
r1---1591371894.125551
r2---1591371894.137864
initrtt19.844000
r1---1591371895.682997
r2---1591371895.691428
initrtt19.724000
r1---1591371897.216336
r2---1591371897.225662
initrtt20.690000
r1---1591371898.910680
r2---1591371898.921857
initrtt19.920000
r1---1591371900.422910
r2---1591371900.431304
initrtt18.376000
r1---1591371902.003337
r2---1591371902.003438
initrtt19.815000
r1---1591371903.683240
r2---1591371903.684891
initrtt26.479000
r1---1591371905.362174
r2---1591371905.363918
initrtt19.677000
r1---1591371907.035740
r2---1591371907.044212
initrtt25.073000
r1---1591371908.700347
r2---1591371908.703104
initrtt26.533000
r1---1591371910.346245
r2---1591371910.348140
initrtt24.918000
r1---1591371912.102517
r2---1591371912

6
6
conn:37272
7
7
conn:37318
8
8
conn:37332
9
9
conn:37344
10
10
conn:37356
11
11
conn:37370
12
12
conn:37378
13
13
conn:37392
14
14
conn:37400
15
15
conn:37408
16
16
conn:37416
17
17
conn:37426
18
18
conn:37450
19
19
conn:37458
20
20
conn:37466
21
21
conn:37476
22
22
conn:37482
23
23
conn:37494
24
24
conn:37502
25
25
conn:37516
26
26
conn:37524
27
27
conn:37532
28
28
conn:37566
29
29
conn:37586
30
30
conn:37594
31
31
conn:37610
32
32
conn:37618
33
33
conn:37626
34
34
conn:37632
35
35
conn:37642
36
36
conn:37648
37
37
conn:37666
38
38
conn:37710
39
39
conn:37750
40
40
conn:37774
41
41
conn:37786
42
42
conn:37796
43
43
conn:37804
44
44
conn:37812
45
45
conn:37820
46
46
conn:37828
47
47
conn:37836
48
48
conn:37844
49
49
conn:37852
50
50
conn:37862
51
51
conn:37872
52
52
conn:37888
53
53
conn:37926
54
54
conn:37948
55
55
conn:37956
56
56
conn:37968
57
57
conn:37980
58
58
conn:38010
59
59
conn:38044
60
2
60
initrtt28.066000
r1---1591336947.735805
r2---1591336947.736076
initrtt109.074000
r

...

1
1
conn:48590
2
2
conn:48598
3
3
conn:48612
4
4
conn:48640
5
5
conn:48656
6
6
conn:48672
7
7
conn:48692
8
8
conn:48712
9
9
conn:48732
10
10
conn:48740
11
11
conn:48750
12
12
conn:48758
13
13
conn:48766
14
14
conn:48774
15
15
conn:48782
16
16
conn:48790
17
17
conn:48798
18
18
conn:48806
19
19
conn:48814
20
20
conn:48824
21
21
conn:48834
22
22
conn:48842
23
23
conn:48850
24
24
conn:48860
25
25
conn:48868
26
26
conn:48876
27
27
conn:48882
28
28
conn:48892
29
29
conn:48900
30
30
conn:48908
31
31
conn:48916
32
32
conn:48924
33
33
conn:48932
34
34
conn:48940
35
35
conn:48948
36
36
conn:48956
37
37
conn:48964
38
38
conn:48972
39
39
conn:48980
40
40
conn:48984
41
41
conn:48996
42
42
conn:49004
43
43
conn:49008
44
44
conn:49016
45
45
conn:49024
46
46
conn:49032
47
47
conn:49040
48
48
conn:49052
49
49
conn:49060
50
50
conn:49068
51
51
conn:49076
52
52
conn:49084
53
53
conn:49092
54
54
conn:49100
55
55
conn:49108
56
56
conn:49116
57
57
conn:49124
58
58
conn:49132
59
59
conn:49140
60
1
60
initrtt

initrtt22.976000
r1---1591341409.441189
r2---1591341409.450179
initrtt19.435000
r1---1591341411.389727
r2---1591341411.405711
initrtt21.391000
r1---1591341414.623165
r2---1591341414.643230
initrtt26.146000
r1---1591341417.717646
r2---1591341417.732511
initrtt22.341000
r1---1591341419.445122
r2---1591341419.462958
initrtt15.601000
r1---1591341422.039804
r2---1591341422.048999
initrtt27.509000
r1---1591341423.520011
r2---1591341423.520121
initrtt17.503000
r1---1591341426.594040
r2---1591341426.598992
initrtt26.923000
r1---1591341428.481664
r2---1591341428.481773
initrtt26.377000
r1---1591341430.518515
r2---1591341430.518697
initrtt21.124000
r1---1591341432.318932
r2---1591341432.333156
initrtt11.094000
r1---1591341434.220688
r2---1591341434.220810
initrtt34.486000
r1---1591341435.713537
r2---1591341435.731960
initrtt26.585000
r1---1591341437.644110
r2---1591341437.660538
initrtt32.513000
r1---1591341439.652697
r2---1591341439.667688
initrtt26.755000
r1---1591341441.801931
r2---1591341441

1
1
conn:56178
2
2
conn:56186
3
3
conn:56190
4
4
conn:56206
5
5
conn:56216
6
6
conn:56224
7
7
conn:56232
8
8
conn:56238
9
9
conn:56248
10
10
conn:56252
11
11
conn:56264
12
12
conn:56268
13
13
conn:56278
14
14
conn:56288
15
15
conn:56294
16
16
conn:56300
17
17
conn:56308
18
18
conn:56320
19
19
conn:56328
20
20
conn:56332
21
21
conn:56342
22
22
conn:56348
23
23
conn:56356
24
24
conn:56364
25
25
conn:56372
26
26
conn:56380
27
27
conn:56388
28
28
conn:56396
29
29
conn:56404
30
30
conn:56412
31
31
conn:56422
32
32
conn:56430
33
33
conn:56440
34
34
conn:56448
35
35
conn:56456
36
36
conn:56464
37
37
conn:56470
38
38
conn:56478
39
39
conn:56482
40
40
conn:56492
41
41
conn:56502
42
42
conn:56510
43
43
conn:56514
44
44
conn:56524
45
45
conn:56534
46
46
conn:56538
47
47
conn:56546
48
48
conn:56558
49
49
conn:56568
50
50
conn:56574
51
51
conn:56584
52
52
conn:56592
53
53
conn:56598
54
54
conn:56608
55
55
conn:56612
56
56
conn:56620
57
57
conn:56628
58
58
conn:56636
59
59
conn:56644
60
0
60
initrtt

initrtt34.270000
r1---1591283795.255864
r2---1591283795.292104
initrtt55.593000
r1---1591283799.104557
r2---1591283799.142580
initrtt18.899000
r1---1591283803.124184
r2---1591283803.148433
initrtt30.328000
r1---1591283807.122103
r2---1591283807.157848
initrtt35.647000
r1---1591283811.081496
r2---1591283811.118424
initrtt44.728000
r1---1591283814.210326
r2---1591283814.239242
initrtt57.410000
r1---1591283815.793286
r2---1591283815.828136
initrtt33.730000
r1---1591283819.438159
r2---1591283819.467959
initrtt35.335000
r1---1591283823.103373
r2---1591283823.144133
initrtt45.221000
r1---1591283827.432466
r2---1591283827.465420
initrtt39.258000
r1---1591283830.159505
r2---1591283830.174598
initrtt19.213000
r1---1591283831.863893
r2---1591283831.887542
initrtt32.826000
r1---1591283834.642985
r2---1591283834.664794
initrtt30.827000
r1---1591283837.467322
r2---1591283837.479843
initrtt36.288000
r1---1591283839.947876
r2---1591283839.968673
initrtt45.317000
r1---1591283842.676900
r2---1591283842

1
1
conn:35652
2
2
conn:35664
3
3
conn:35672
4
4
conn:35680
5
5
conn:35688
6
6
conn:35702
7
7
conn:35710
8
8
conn:35720
9
9
conn:35732
10
10
conn:35736
11
11
conn:35744
12
12
conn:35752
13
13
conn:35762
14
14
conn:35770
15
15
conn:35778
16
16
conn:35786
17
17
conn:35794
18
18
conn:35800
19
19
conn:35806
20
20
conn:35818
21
21
conn:35822
22
22
conn:35834
23
23
conn:35842
24
24
conn:35846
25
25
conn:35854
26
26
conn:35866
27
27
conn:35874
28
28
conn:35882
29
29
conn:35888
30
30
conn:35894
31
31
conn:35902
32
32
conn:35910
33
33
conn:35918
34
34
conn:35926
35
35
conn:35938
36
36
conn:35944
37
37
conn:35954
38
38
conn:35962
39
39
conn:35970
40
40
conn:35978
41
41
conn:35986
42
42
conn:35994
43
43
conn:36002
44
44
conn:36010
45
45
conn:36018
46
46
conn:36026
47
47
conn:36034
48
48
conn:36042
49
49
conn:36048
50
50
conn:36056
51
51
conn:36064
52
52
conn:36072
53
53
conn:36080
54
54
conn:36090
55
55
conn:36096
56
56
conn:36106
57
57
conn:36114
58
58
conn:36126
59
59
conn:36136
60
1
60
initrtt